### Импорты

In [1]:
import os
import pandas as pd
import datetime
import warnings
import catboost
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [2]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

Добавил две фичи: kda_ratio, level_ratio. Это отношение среднего KDA и уровней radiant vs dire. Должно послужить хорошей фичёй в теории 

In [12]:
# Считаем таблицу с признаками и таргетами
df_train_features_extended = pd.read_csv('../data/df_train_features_extended.csv', index_col='match_id_hash')
df_train_targets = pd.read_csv('../data/df_train_targets.csv', index_col='match_id_hash')


Я решил попробовать использовать ансамбль. Катбуст - поскольку он из коробки работает с категориальными переменными. Ну и логистическая регрессия, поскольку она не основана на деревьях и наверное хорошо сочтётся с моделью на деревьях

Определил несколько переменных как категориальные, поэтому их закодируем. Остальные шкалируем, чтобы можно было это всё дело обработать логистической регрессией.

Поитогу подумал, ну раз уш шкалируем, то и для катбуста не помешает...


In [14]:
cat_cols = df_train_features_extended.filter(like = 'firstblood_claimed').columns.to_list()
cat_cols.append('lobby_type')
num_cols = set(df_train_features_extended.columns).difference(set(cat_cols))
num_cols = list(num_cols)
preprocessor = ColumnTransformer(transformers=[
    ("scaler", StandardScaler(), num_cols),
    ("ohe", OneHotEncoder(drop="first"), cat_cols)
])

In [15]:
catboost_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("Catboost", catboost.CatBoostClassifier(verbose=0, random_seed=SEED))])
regression_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("LR", LogisticRegression(max_iter = 5000, n_jobs=-1))
])
base_models = [("Cat", catboost_pipeline), ("Log_reg", regression_pipeline)]

In [63]:
y = df_train_targets["radiant_win"].values.astype("int8")
voting_soft = VotingClassifier(base_models, voting='soft')
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)
scores = cross_val_score(voting_soft, df_train_features_extended, y, cv=cv, scoring="roc_auc", n_jobs=-1)
print(f"ROC-AUC на кросс-валидации для новых признаков: {scores.mean()}")

ROC-AUC на кросс-валидации для новых признаков: 0.8120130219651364


In [64]:
voting_soft = VotingClassifier(base_models, voting='soft')
voting_soft.fit(df_train_features_extended, y)

VotingClassifier(estimators=[('Cat',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('scaler',
                                                                                StandardScaler(),
                                                                                ['d3_obs_placed',
                                                                                 'd5_level',
                                                                                 'd5_roshans_killed',
                                                                                 'd4_lh',
                                                                                 'r1_xp',
                                                                                 'r4_assists',
                                                                                 'r4_camps_stacked',
                                                                                 'd3_stuns',
                                                                                 'd1_teamfight_participation',
                                                                                 'r2_creeps_stacked',
                                                                                 'd4_y',
                                                                                 'r2_sen_placed',
                                                                                 'd4_teamfight_participation',
                                                                                 'r5_y',
                                                                                 'r5_heal...
                                                                                OneHotEncoder(drop='first'),
                                                                                ['r1_firstblood_claimed',
                                                                                 'r2_firstblood_claimed',
                                                                                 'r3_firstblood_claimed',
                                                                                 'r4_firstblood_claimed',
                                                                                 'r5_firstblood_claimed',
                                                                                 'd1_firstblood_claimed',
                                                                                 'd2_firstblood_claimed',
                                                                                 'd3_firstblood_claimed',
                                                                                 'd4_firstblood_claimed',
                                                                                 'd5_firstblood_claimed',
                                                                                 'lobby_type'])])),
                                              ('LR',
                                               LogisticRegression(max_iter=5000,
                                                                  n_jobs=-1))]))],
                 voting='soft')

In [65]:
df_test_features_extended = pd.read_csv('../data/df_test_features_extended.csv')
y_test_pred = voting_soft.predict_proba(df_test_features_extended)[:, 1]
df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred},
                                 index=df_test_features_extended.index)

  0%|          | 0/31698 [00:00<?, ?it/s]

In [66]:
submission_filename = "../data/submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Файл посылки сохранен, как: submission_2024-04-05_11-11-58.csv
